In [ ]:
#library
import rasterio
import rasterio
from rasterio.features import rasterize
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import csv

In [ ]:
#Validation Dataset
Validation_polygon='you shapefile polygon'
RBF_raster='your RFB generated geotiff layer'

In [ ]:
#accuracy assessment
def calculate_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    return np.sum(intersection) / np.sum(union)

def calculate_agreement_rate(tp, fp, fn):
    return tp / (tp + fn) * 100

def calculate_commission_rate(tp, fp, fn):
    return fp / (tp + fn) * 100

def calculate_ommission_rate(tp, fp, fn):
    return fn / (tp + fn) * 100

# Load the predicted footprint raster
with rasterio.open(RBF_raster) as src:
    predicted_raster = src.read(1)
    transform = src.transform
    crs = src.crs

# Load the ground truth shapefile
ground_truth = gpd.read_file(Validation_polygon)

# Ensure the ground truth is in the same CRS as the predicted raster
if ground_truth.crs != crs:
    ground_truth = ground_truth.to_crs(crs)

# Rasterize the ground truth polygons
shapes = [(geom, 1) for geom in ground_truth.geometry]
ground_truth_raster = rasterize(shapes, out_shape=predicted_raster.shape, transform=transform)

# Flatten the arrays and convert to binary (0 or 1)
y_true = ground_truth_raster.flatten() > 0
y_pred = predicted_raster.flatten() == 5  # Change here: we're looking for pixels with value 5

# Calculate metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
iou = calculate_iou(y_true, y_pred)

# Calculate TP, FP, FN
tp = np.sum(np.logical_and(y_true, y_pred))
fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))
fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))

# Calculate agreement rate (TPrate)
agreement_rate = calculate_agreement_rate(tp, fp, fn)
commission_rate = calculate_commission_rate(tp, fp, fn)
ommission_rate = calculate_ommission_rate(tp, fp, fn)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"IoU: {iou:.4f}")
print(f"Agreement Rate (TPrate): {agreement_rate:.4f}%")
print(f"Commission Rate (FPrate): {commission_rate:.4f}%")
print(f"Ommision Rate (FNrate): {ommission_rate:.4f}%")

# Save metrics to CSV
csv_filename = 'your saving path'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Metric', 'Value'])
    writer.writerow(['Precision', f"{precision:.4f}"])
    writer.writerow(['Recall', f"{recall:.4f}"])
    writer.writerow(['F1 Score', f"{f1:.4f}"])
    writer.writerow(['IoU', f"{iou:.4f}"])
    writer.writerow(['Agreement Rate (TPrate)', f"{agreement_rate:.4f}%"])

print(f"Metrics saved to {csv_filename}")